#### Lectura de los datos contenidos en los archivos csv y generación de las tablas en mysql

In [1]:
from Configuración_BD import engine
import pandas as pd

db = engine.connect()

df_alumnos = pd.read_csv("Bases de datos/Alumnos.csv",sep=",") # lee la base de datos "alumnos" que está en formato csv y lo guarda en la variable df
df_profesores = pd.read_csv("Bases de datos/Profesores.csv",sep=",") # lo mismo con "profesores"
df_cursos_profesores = pd.read_csv("Bases de datos/Cursos_profesores.csv",sep=",") # lo mismo con "cursos_profesores"


try:
	df_alumnos.to_sql("alumnos", db, if_exists="fail") # convierte el dataframe (df) de alumnos a sql y crea la tabla profes 
	df_profesores.to_sql("profesores", db, if_exists="fail") # lo mismo con "profesores"
	df_cursos_profesores.to_sql("cursos_profesores", db, if_exists="fail") # lo mismo con "cursos_profesores" 

except ValueError as vx:
	print(vx)
except Exception as ex:
	print (ex)
else:
	print("Tablas creadas con éxito")
finally:	
	db.close()

Tablas creadas con éxito


#### Lectura de las tablas y conversión del dataframe a una lista de diccionarios (dataset)

In [2]:
def leer_sql(tabla):    
    df= pd.read_sql(f"select * from {tabla}", engine) # lee x tabla (sql) y lo guarda en la variable df
    dataset=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset)
    db.close()
    return dataset
    
dataset_alumnos=leer_sql("alumnos") # se generan los respectivos dataset
dataset_profesores=leer_sql("profesores")
dataset_cursos_profesores=leer_sql("cursos_profesores")

#### Eliminar las tablas recién generadas

In [3]:
import MySQLdb
def eliminar_tabla(sql): # función para eliminar una tabla determinada
    try:
        connection = MySQLdb.connect(host="127.0.0.1", # conexión a la base de datos
						user="root",
						passwd="Contadores2",
						db="pil_trabajo",
                        port=4000)

        cursor = connection.cursor()
        cursor.execute(sql) # acá se inserta la sentencia sql (drop table x)
        cursor.close()

    except MySQLdb.Error as error:
        print("Error al eliminar la tabla: {}".format(error))
    else:
        print("Tabla eliminada exitosamente")

    finally:
        if connection.ping:
            connection.close()
            print("Conexión con MySQL cerrada")
       
eliminar_tabla("drop table alumnos")
eliminar_tabla("drop table profesores")
eliminar_tabla("drop table cursos_profesores")

Tabla eliminada exitosamente
Conexión con MySQL cerrada
Tabla eliminada exitosamente
Conexión con MySQL cerrada
Tabla eliminada exitosamente
Conexión con MySQL cerrada


#### Eliminar el campo "index" de los dataset

In [4]:
def eliminar_index(dataset): # función para eliminar el campo index del dataset
    for x in dataset: #se recorre el dataset
        del x["index"]   # se elimina la clave "index" de cada registro
                
eliminar_index(dataset_alumnos)
eliminar_index(dataset_profesores)

#### Extraer valores únicos del campo "gender" de los dataset y traducirlos al español

In [5]:
def traduccion_gender(dataset): # función para traducir los valores del campo "gender"
    lista=[]
    for x in dataset: # recorremos el dataset
        if x["gender"] not in lista: # si el el valor de gender no está en la lista, lo agregamos a dicha lista 
            lista.append(x["gender"])    # de esta forma se obtienen los valores únicos que se utilizan a continuación

    for x in dataset:
        if x["gender"]=="Female":
            x["gender"]="Femenino"
        elif x["gender"]=="Male":
            x["gender"]="Masculino"
        elif x["gender"]=="Non-binary":
            x["gender"]="No-binario"
        elif x["gender"]=="Polygender":
            x["gender"]="Poligénero"
        elif x["gender"]=="Genderqueer":
            x["gender"]="Género queer"
        elif x["gender"]=="Agender":
            x["gender"]="Agénero"
        elif x["gender"]=="Bigender":
            x["gender"]="Bigénero"
        elif x["gender"]=="Genderfluid":
            x["gender"]="Género fluido"   

traduccion_gender(dataset_profesores)
traduccion_gender(dataset_alumnos)

#### Convertir los campos "personal_id" e "instructor" de str a número

In [6]:
def conversion_dni_numero(dataset): # función para convertir el formato de los campos personal_id e instructor, que serían el dni, de str a número
    for x in dataset: # recorremos el dataset
        if "personal_id" in x: # seleccionamos la clave personal_id                  
            j=x["personal_id"].replace('.','') # eliminamos los puntos mediante el método replace
            x["personal_id"]=j # asignamos un nuevo valor a la clave personal_id
        if "instructor" in x: # lo mismo pero con el campo instructor
            j=x["instructor"].replace('.','')
            x["instructor"]=j    
    
conversion_dni_numero(dataset_cursos_profesores)
conversion_dni_numero(dataset_alumnos)
conversion_dni_numero(dataset_profesores)

#### Eliminar registros repetidos en su totalidad o que tienen un campo repetido 

In [7]:

def eliminar_registros_repetidos_entotalidad(dataset): # función para eliminar aquellos registros que se repiten en su totalidad
    global nuevo_dataset  # hacemos la variable nuevo_dataset sea global para que exista por fuera de la función   
    lista_unica = set()   # generamos un conjunto para almacenar los registros únicos pero en forma de tuplas
    nuevo_dataset = []    # definimos una lista que va a almacenar los registros únicos en forma de diccionarios
    repetidos = []  # lista que va a almacenar los registros eliminados

    for j in dataset: # recorremos el dataset
        t = tuple(j.items()) # convierte a la lista de tuplas de los pares clave-valor de cada registro en una tupla de tuplas
        if t not in lista_unica: # si ese registro en forma de tupla de tuplas no está en el set lo agrega y a su 
                                 # vez agrega el registro en forma de diccionario a la lista nuevo_dataset
            lista_unica.add(t)
            nuevo_dataset.append(j)
        else:                    # si no, se lo agrega al registro en forma de diccionario a la lista de repetidos
            repetidos.append(j)      
        

def eliminar_registros_repetidos_uncampo(dataset,campo): # esta función sirve para eliminar aquellos registros que se repiten parcialmente, es decir en un campo determinado
    valores_unicos = [] # crea una lista para almacenar todos los valores únicos del campo seleccionado
    registros_repetidos = [] # crea una lista para almacenar todos los registros en los cuales en el campo seleccionado tengan un valor repetido
    for x in dataset: # recorremos el dataset
        for i,j in x.items():      # transformamos cada registro (diccionario) en una lista de tuplas 
            if i==campo:    # seleccionamos la clave que sea igual al campo elegido      
                if j not in valores_unicos:      # si su valor no está en la lista, lo agregamos  
                    valores_unicos.append(j)              
                else:                         
                    registros_repetidos.append(x)   # si no agregamos el diccionario a la lista de registros repetidos     
    for x in dataset:   # se elimina del dataset todos aquellos registros que se encuentren en la lista de registros_repetidos
        if x in registros_repetidos:
            del dataset[dataset.index(x)]   
    print(registros_repetidos) 


def buscar_registros_repetidos_uncampo(dataset,campo): # esta función es por si no se quiere eliminar aquellos registros con 
                                                       # campos repetidos, sino que solamente se los quiere encontrar para quizás modificar el 
                                                       # campo antes que borrar todo el registro
    valores_unicos = [] 
    registros_repetidos = [] 
    for x in dataset:
        for i,j in x.items():      
            if i==campo:          
                if j not in valores_unicos:        
                    valores_unicos.append(j)              
                else:                         
                    registros_repetidos.append(x)     
    print(registros_repetidos)


#### Crear las tablas del archivo tablas

In [8]:
from Tablas import Base,engine,Profesores,Alumnos,Carreras,Profesores_Carreras,Alumnos_Carreras, Facultades,Ramas,Campus,Ubicacion,Provincias,Ciudades,Municipios,Paises,Genero

db = engine.connect()

Base.metadata.create_all(engine) # creación de las tablas


#### Insertar datos en tablas

In [9]:
from sqlalchemy.orm import sessionmaker
from Tablas import Base,engine,Profesores,Alumnos,Carreras,Profesores_Carreras,Alumnos_Carreras, Facultades,Ramas,Campus,Provincias,Ciudades,Municipios,Paises,Ubicacion,Genero

Session = sessionmaker(bind = engine)
session =  Session()

# todos los dataset que van a ser rellenados con datos para después ser insertados en sus respectivas tablas

dataset_facultades=[] 
dataset_campus=[] 
dataset_ramas=[] 
dataset_genero=[] 
dataset_ubicacion=[] 
dataset_profesores2=[] 
dataset_alumnos2=[] 
dataset_carreras=[] 
dataset_profesores_carreras=[] 
dataset_alumnos_carreras=[]
dataset_ciudades=[] 
dataset_municipios=[] 
dataset_provincias=[] 
dataset_paises=[]

def insertar_datos(dataset,tabla): # función para insertar los datos contenidos en cada dataset a su tabla correspondiente
    lista = []

    for x in dataset:
        lista.append(tabla(**x))
    session.add_all(lista)
    session.commit()
    session.close()

# TABLAS: RAMAS, FACULTADES, CAMPUS Y GENERO

def generar_dataset_RFCG(dataset_definitivo,dataset_original,campo,clave): #para generar los dataset de las tablas facultades, campus, ramas y genero
    valores_unicos=[]      
    for x in dataset_original: # recorremos el "dataset_original", que sería uno de los 3 dataset generados al principio a partir de los archivos csv
        if x[campo] not in valores_unicos: # si el valor del campo seleccionado no está en la lista de valores únicos lo agregamos
            valores_unicos.append(x[campo])
            dataset_definitivo.append({clave:(x[campo])}) # y agregamos ese valor único junto con una clave determinada, todo en forma de un 
                                                        # diccionario, al dataset_definitivo, el que se va a usar 
                                                        # después para insertar los datos       

generar_dataset_RFCG(dataset_facultades,dataset_cursos_profesores,"institute","nombre") # generamos los dataset
generar_dataset_RFCG(dataset_ramas,dataset_cursos_profesores,"branch","nombre")
generar_dataset_RFCG(dataset_campus,dataset_cursos_profesores,"campus","nombre")
generar_dataset_RFCG(dataset_genero,dataset_alumnos,"gender","tipo")

insertar_datos(dataset_ramas,Ramas) # insertamos los datos de los dataset en sus respectivas tablas
insertar_datos(dataset_facultades,Facultades)
insertar_datos(dataset_campus,Campus)
insertar_datos(dataset_genero,Genero)

# TABLAS: CIUDADES, MUNICIPIOS, PROVINCIAS Y PAÍSES

def generar_dataset_CMPP(dataset_definitivo,campo): #para generar los dataset de las tablas ciudades, provincias, 
                                                    # municipios y países. El método es el mismo que en los anteriores con la diferencia
                                                    # de que acá hay que recorrer dos dataset de los originales en vez de uno, 
                                                    # ya que los campos city, town, state y country
                                                    # se encuentran en ambos dataset    
    valores_unicos=[]
    for x in dataset_alumnos:        
            if x[campo] not in valores_unicos:
                valores_unicos.append(x[campo])
                dataset_definitivo.append({"nombre":(x[campo])})  
    for x in dataset_profesores:        
            if x[campo] not in valores_unicos:
                valores_unicos.append(x[campo])
                dataset_definitivo.append({"nombre":(x[campo])})       
    
generar_dataset_CMPP(dataset_ciudades,"city")
generar_dataset_CMPP(dataset_municipios,"town")
generar_dataset_CMPP(dataset_provincias,"state")
generar_dataset_CMPP(dataset_paises,"country")

insertar_datos(dataset_ciudades,Ciudades)
insertar_datos(dataset_municipios,Municipios)
insertar_datos(dataset_provincias,Provincias)
insertar_datos(dataset_paises,Paises)

# TABLA: UBICACION

def generar_dataset_ubicacion(): # función para generar el dataset de la tabla ubicación, que tiene como clave foráneas a los id de las tablas ciudades, municipios y provincias
    
    for x in dataset_alumnos:
        ciudades_id=session.query(Ciudades).filter(Ciudades.nombre==x["city"]) # traigo todos los registros de la tabla ciudades cuyo campo nombre (el valor) sea igual al valor del campo "city" del dataset_alumnos
        municipios_id=session.query(Municipios).filter(Municipios.nombre==x["town"]) # lo mismo pero con el campo "town" (municipio)
        provincias_id=session.query(Provincias).filter(Provincias.nombre==x["state"]) # lo mismo pero con el campo "state" (provincia)
        paises_id=session.query(Paises).filter(Paises.nombre==x["country"]) # lo mismo pero con el campo "country" (provincia)
        for row in ciudades_id: # recorro cada uno de los campos de los registros obtenidos anteriormente
            dataset_ubicacion.append({"ciudades_id":row.id}) #agrego al dataset un diccionario cuyo valor se corresponde con el campo id de row
        for row in municipios_id: 
            dataset_ubicacion[dataset_alumnos.index(x)].update({"municipios_id":row.id}) # la misma operación que la anterior para el campo municipios, variando que en vez de agregar un nuevo diccionario modifico el ya existento agregándole un nuevo par clave-valor
        for row in provincias_id:
            dataset_ubicacion[dataset_alumnos.index(x)].update({"provincias_id":row.id})
        for row in paises_id:
            dataset_ubicacion[dataset_alumnos.index(x)].update({"paises_id":row.id})
            
    for x in dataset_profesores: # repetimos todo lo hecho anteriormente para el dataset_profesores, ya que hay ciudades, municipios y provincias exclusivos de este dataset que no están en el de alumnos
        ciudades_id=session.query(Ciudades).filter(Ciudades.nombre==x["city"])
        municipios_id=session.query(Municipios).filter(Municipios.nombre==x["town"])
        provincias_id=session.query(Provincias).filter(Provincias.nombre==x["state"])
        paises_id=session.query(Paises).filter(Paises.nombre==x["country"])
        for row in ciudades_id:
            dataset_ubicacion.append({"ciudades_id":row.id})
        for row in municipios_id:
            dataset_ubicacion[dataset_profesores.index(x)+1000].update({"municipios_id":row.id}) # el +1000 en el index es para continuar desde el último registro del paso anterior, que inserta 1000 registros
        for row in provincias_id:
            dataset_ubicacion[dataset_profesores.index(x)+1000].update({"provincias_id":row.id})
        for row in paises_id:
            dataset_ubicacion[dataset_profesores.index(x)+1000].update({"paises_id":row.id})
   
generar_dataset_ubicacion()

#Luego procedo a eliminar los registros repetidos del dataset_ubicacion con la función eliminar_registros_repetidos_entotalidad #

eliminar_registros_repetidos_entotalidad(dataset_ubicacion)
dataset_ubicacion=nuevo_dataset # Acá realizo una copia del nuevo_dataset generado con la función eliminar_registros, para sobrescribir los registros de dataset_ubicacion con el fin de hacer efectivos los cambios realizados con la función, de otra forma dataset_ubicacion permance sin alterar al ser una variable definida por fuera de la función

insertar_datos(dataset_ubicacion,Ubicacion) 

# TABLA: CARRERAS

def generar_dataset_carreras(): # función para generar el dataset de la tabla carreras que tiene como claves foráneas los id de ramas, facultades y campus          
    for x in dataset_cursos_profesores:       
        dataset_carreras.append({"nombre":x["program"]})    
        ramas_id=session.query(Ramas).filter(Ramas.nombre==x["branch"]) # traigo todos los registros de la tabla ciudades cuyo campo nombre (el valor) sea igual al valor del campo "branch" del dataset_cursos_profesores
        facultades_id=session.query(Facultades).filter(Facultades.nombre==x["institute"]) # lo mismo pero con el campo "institute" (municipio)
        campus_id=session.query(Campus).filter(Campus.nombre==x["campus"]) # lo mismo pero con el campo "campus" (provincia)
        for row in ramas_id: # recorro cada uno de los campos de los registros obtenidos anteriormente
            dataset_carreras[dataset_cursos_profesores.index(x)].update({"ramas_id":row.id}) #agrego al dataset un diccionario cuyo valor se corresponde con el campo id de row
        for row in facultades_id: 
            dataset_carreras[dataset_cursos_profesores.index(x)].update({"facultades_id":row.id}) # la misma operación que la anterior para el campo municipios, variando que en vez de agregar un nuevo diccionario modifico el ya existento agregándole un nuevo par clave-valor
        for row in campus_id:
            dataset_carreras[dataset_cursos_profesores.index(x)].update({"campus_id":row.id})

generar_dataset_carreras()

# Misma operación que en el dataset de ubicacion

eliminar_registros_repetidos_entotalidad(dataset_carreras)
dataset_carreras=nuevo_dataset

insertar_datos(dataset_carreras,Carreras)

# TABLAS: ALUMNOS Y PROFESORES

def generar_dataset_AP(dataset_original,dataset_definitivo):  # Para generar los dataset de las tablas alumnos y profesores, que tienen como claves foráneas los id de las tablas ubicacion y carreras         
    for x in dataset_original: # recorremos el dataset_original      
        dataset_definitivo.append({"dni":x["personal_id"]})  # agregamos al dataset_definitivo el campo dni con el valor del campo "personal_id" del dataset_original
        dataset_definitivo[dataset_original.index(x)].update({"nombre":x["first_name"]}) # agregamos el campo nombre
        dataset_definitivo[dataset_original.index(x)].update({"apellido":x["last_name"]}) # agregamos el campo apellido
        dataset_definitivo[dataset_original.index(x)].update({"email":x["email"]}) # agregamos el campo email
        dataset_definitivo[dataset_original.index(x)].update({"pais":x["country"]}) # agregamos el campo pais
        dataset_definitivo[dataset_original.index(x)].update({"fecha_nacimiento":x["birthdate"]}) # agregamos el campo fecha de nacimiento

        ciudad_alumno=session.query(Ciudades).filter(Ciudades.nombre==x["city"]) # para el campo ubicacion_id primero hay que obtener el registro de la tabla ciudades que se corresponde con el alumno de cada registro del dataset_original
        for row in ciudad_alumno: # dentro de ese registro nos quedamos con el id
            id_ciudad_alumno=row.id 
        municipio_alumno=session.query(Municipios).filter(Ciudades.nombre==x["town"]) # lo mismo pero con municipios
        for row in municipio_alumno:
            id_municipio_alumno=row.id
        provincia_alumno=session.query(Provincias).filter(Ciudades.nombre==x["state"]) # lo mismo pero con provincias
        for row in provincia_alumno:
            id_provincia_alumno=row.id
        ubicacion_id=session.query(Ubicacion).filter(Ubicacion.ciudades_id==id_ciudad_alumno and Ubicacion.municipios_id==id_municipio_alumno and Ubicacion.provincias_id==id_provincia_alumno) # traigo todos los registros de la tabla ubicacion en donde los id de ciudades, municipios y provincias coincidan con los id obtenidos anteriormente
        for row in ubicacion_id: # recorro cada uno de los registros obtenidos anteriormente, que en realidad es un solo registro
            dataset_definitivo[dataset_original.index(x)].update({"ubicacion_id":row.id}) #finalmente agrega el campo ubicacion_id con el id del registro obtenido 
        genero_id=session.query(Genero).filter(Genero.tipo==x["gender"]) # agrego el último campo que es genero_id
        for row in genero_id:
            dataset_definitivo[dataset_original.index(x)].update({"genero_id":row.id})

generar_dataset_AP(dataset_profesores,dataset_profesores2)
generar_dataset_AP(dataset_alumnos,dataset_alumnos2)

insertar_datos(dataset_alumnos2,Alumnos)
insertar_datos(dataset_profesores2,Profesores)

# TABLA: ALUMNOS_CARRERAS


def generar_dataset_alumnos_carreras(): # función para generar el dataset correspondiente a la tabla carreras_alumnos
    for x in dataset_alumnos:
        alumno=session.query(Alumnos).filter(Alumnos.dni==x["personal_id"]) # obtengo aquellos registros de la tabla alumnos en donde el valor del dni sea igual al del personal_id del registros seleccionado del dataset_original (dataset_alumnos)
        carrera=session.query(Carreras).filter(Carreras.nombre==x["program"]) # lo mismo pero con el nombre de la carrera
        for row in alumno:            
            alumno_id=row.id # obtenemos el id del registro seleccionado de alumnos
        for row in carrera:
            carrera_id=row.id # obtenemos el id del registro seleccionado de carreras
        dataset_alumnos_carreras.append({"alumno_id":alumno_id,"carrera_id":carrera_id}) # agregamos el diccionario

generar_dataset_alumnos_carreras()

insertar_datos(dataset_alumnos_carreras,Alumnos_Carreras)

# TABLA: PROFESORES_CARRERAS

def generar_dataset_profesores_carreras(): # función para generar el dataset correspondiente a la tabla carreras_profesores, funciona igual que el de carreras_alumnos el método
    for x in dataset_cursos_profesores:
        profe=session.query(Profesores).filter(Profesores.dni==x["instructor"])
        carrera=session.query(Carreras).filter(Carreras.nombre==x["program"])
        for row in profe:            
            profe_id=row.id
        for row in carrera:
            carrera_id=row.id
        dataset_profesores_carreras.append({"profesor_id":profe_id,"carrera_id":carrera_id})
        
generar_dataset_profesores_carreras()

insertar_datos(dataset_profesores_carreras,Profesores_Carreras)

C:\Users\julia\AppData\Local\Temp\ipykernel_3424\1129181525.py:161: SAWarning: SELECT statement has a cartesian product between FROM element(s) "ciudades" and FROM element "municipios".  Apply join condition(s) between each element to resolve.
  for row in municipio_alumno:
C:\Users\julia\AppData\Local\Temp\ipykernel_3424\1129181525.py:164: SAWarning: SELECT statement has a cartesian product between FROM element(s) "provincias" and FROM element "ciudades".  Apply join condition(s) between each element to resolve.
  for row in provincia_alumno:


#### Exportar tablas a csv

In [10]:
def exportar_csv(tabla,nombre_archivo):
    df= pd.read_sql(f"select * from {tabla}", engine) # primero obtenemos el dataframe a partir de mysql
    df.to_csv(f"{nombre_archivo}.csv",index=False, encoding="utf-8") # después lo exportamos al formato csv, 
                                                    # eliminando el index que se agrega automáticamente 
                                                    # y codificándolo en utf-8 para su lectura en cualquier sistema 
                                                    # operativo